In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
import time

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
data = pd.read_csv('media prediction and its cost.csv')
data.head()

,food_category,food_department,food_family,store_sales(in millions),store_cost(in millions),unit_sales(in millions),promotion_name,sales_country,marital_status,gender,...,grocery_sqft,frozen_sqft,meat_sqft,coffee_bar,video_store,salad_bar,prepared_food,florist,media_type,cost
0,Breakfast Foods,Frozen Foods,Food,7.36,2.7232,4.0,Bag Stuffers,USA,M,F,...,18670.0,5415.0,3610.0,1.0,1.0,1.0,1.0,1.0,"Daily Paper, Radio",126.62
1,Breakfast Foods,Frozen Foods,Food,5.52,2.5944,3.0,Cash Register Lottery,USA,M,M,...,18670.0,5415.0,3610.0,1.0,1.0,1.0,1.0,1.0,"Daily Paper, Radio",59.86
2,Breakfast Foods,Frozen Foods,Food,3.68,1.3616,2.0,High Roller Savings,USA,S,F,...,18670.0,5415.0,3610.0,1.0,1.0,1.0,1.0,1.0,"Daily Paper, Radio",84.16
3,Breakfast Foods,Frozen Foods,Food,3.68,1.1776,2.0,Cash Register Lottery,USA,M,F,...,18670.0,5415.0,3610.0,1.0,1.0,1.0,1.0,1.0,In-Store Coupon,95.78
4,Breakfast Foods,Frozen Foods,Food,4.08,1.4280,3.0,Double Down Sale,USA,M,M,...,18670.0,5415.0,3610.0,1.0,1.0,1.0,1.0,1.0,Radio,50.79


In [3]:
data.shape

(60428, 40)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60428 entries, 0 to 60427
Data columns (total 40 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   food_category               60428 non-null  object 
 1   food_department             60428 non-null  object 
 2   food_family                 60428 non-null  object 
 3   store_sales(in millions)    60428 non-null  float64
 4   store_cost(in millions)     60428 non-null  float64
 5   unit_sales(in millions)     60428 non-null  float64
 6   promotion_name              60428 non-null  object 
 7   sales_country               60428 non-null  object 
 8   marital_status              60428 non-null  object 
 9   gender                      60428 non-null  object 
 10  total_children              60428 non-null  float64
 11  education                   60428 non-null  object 
 12  member_card                 60428 non-null  object 
 13  occupation                  604

In [5]:
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.max_colwidth', None)  # Don't truncate long columns
pd.set_option('display.expand_frame_repr', False)  # Don't wrap lines

In [6]:
data.describe(include='all')

,food_category,food_department,food_family,store_sales(in millions),store_cost(in millions),unit_sales(in millions),promotion_name,sales_country,marital_status,gender,total_children,education,member_card,occupation,houseowner,avg_cars_at home(approx),avg. yearly_income,num_children_at_home,avg_cars_at home(approx).1,brand_name,SRP,gross_weight,net_weight,recyclable_package,low_fat,units_per_case,store_type,store_city,store_state,store_sqft,grocery_sqft,frozen_sqft,meat_sqft,coffee_bar,video_store,salad_bar,prepared_food,florist,media_type,cost
count,60428,60428,60428,60428.000000,60428.000000,60428.000000,60428,60428,60428,60428,60428.000000,60428,60428,60428,60428,60428.000000,60428,60428.000000,60428.000000,60428,60428.000000,60428.000000,60428.000000,60428.000000,60428.000000,60428.000000,60428,60428,60428,60428.000000,60428.000000,60428.000000,60428.000000,60428.000000,60428.000000,60428.000000,60428.000000,60428.000000,60428,60428.000000
unique,45,22,3,NaN,NaN,NaN,49,3,2,2,NaN,5,4,5,2,NaN,8,NaN,NaN,111,NaN,NaN,NaN,NaN,NaN,NaN,5,19,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,NaN
top,Vegetables,Produce,Food,NaN,NaN,NaN,Weekend Markdown,USA,S,F,NaN,Partial High School,Bronze,Professional,Y,NaN,$30K - $50K,NaN,NaN,Hermanos,NaN,NaN,NaN,NaN,NaN,NaN,Supermarket,Tacoma,WA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Daily Paper, Radio",NaN
freq,7440,8521,43284,NaN,NaN,NaN,2330,38892,30355,30942,NaN,18201,33807,19915,36510,NaN,19514,NaN,NaN,1839,NaN,NaN,NaN,NaN,NaN,NaN,26192,5704,19370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6820,NaN
mean,NaN,NaN,NaN,6.541031,2.619460,3.093169,NaN,NaN,NaN,NaN,2.533875,NaN,NaN,NaN,NaN,2.200271,NaN,0.829351,2.200271,NaN,2.115258,13.806433,11.796289,0.558665,0.350434,18.860694,NaN,NaN,NaN,27988.477494,19133.799696,5312.852552,3541.846280,0.612646,0.354157,0.587956,0.587956,0.562603,NaN,99.262366
std,NaN,NaN,NaN,3.463047,1.453009,0.827677,NaN,NaN,NaN,NaN,1.490165,NaN,NaN,NaN,NaN,1.109644,NaN,1.303424,1.109644,NaN,0.932829,4.622693,4.682986,0.496551,0.477110,10.258555,NaN,NaN,NaN,5701.022090,3987.395735,1575.907263,1050.471635,0.487150,0.478261,0.492207,0.492207,0.496069,NaN,30.011257
min,NaN,NaN,NaN,0.510000,0.163200,1.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000,0.000000,NaN,0.500000,6.000000,3.050000,0.000000,0.000000,1.000000,NaN,NaN,NaN,20319.000000,13305.000000,2452.000000,1635.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,50.790000
25%,NaN,NaN,NaN,3.810000,1.500000,3.000000,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,1.000000,NaN,0.000000,1.000000,NaN,1.410000,9.700000,7.710000,0.000000,0.000000,10.000000,NaN,NaN,NaN,23593.000000,16232.000000,4746.000000,3164.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,69.650000
50%,NaN,NaN,NaN,5.940000,2.385600,3.000000,NaN,NaN,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,2.000000,NaN,0.000000,2.000000,NaN,2.130000,13.600000,11.600000,1.000000,0.000000,19.000000,NaN,NaN,NaN,27694.000000,18670.000000,5062.000000,3375.000000,1.000000,0.000000,1.000000,1.000000,1.000000,NaN,98.520000
75%,NaN,NaN,NaN,8.670000,3.484025,4.000000,NaN,NaN,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,3.000000,NaN,1.000000,3.000000,NaN,2.790000,17.700000,16.000000,1.000000,1.000000,28.000000,NaN,NaN,NaN,30797.000000,22123.000000,5751.000000,3834.000000,1.000000,1.000000,1.000000,1.000000,1.000000,NaN,126.620000


In [7]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [8]:
dtypes = data.dtypes

categorical_cols = data.columns[dtypes == 'object'].tolist()
numerical_cols = data.columns[dtypes == 'float64'].tolist()

encoder = OneHotEncoder(handle_unknown='ignore')
encoded_data = encoder.fit_transform(data[categorical_cols])

encoded_df = pd.DataFrame(encoded_data.toarray(), columns=encoder.get_feature_names_out(categorical_cols))

processed_data = pd.concat([encoded_df, data[numerical_cols]], axis=1)

In [9]:
processed_data.shape

(60428, 331)

In [10]:
processed_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60428 entries, 0 to 60427
Data columns (total 331 columns):
 #    Column                                  Dtype  
---   ------                                  -----  
 0    food_category_Baking Goods              float64
 1    food_category_Bathroom Products         float64
 2    food_category_Beer and Wine             float64
 3    food_category_Bread                     float64
 4    food_category_Breakfast Foods           float64
 5    food_category_Candles                   float64
 6    food_category_Candy                     float64
 7    food_category_Canned Anchovies          float64
 8    food_category_Canned Clams              float64
 9    food_category_Canned Oysters            float64
 10   food_category_Canned Sardines           float64
 11   food_category_Canned Shrimp             float64
 12   food_category_Canned Soup               float64
 13   food_category_Canned Tuna               float64
 14   food_category_Carbon

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV

X = processed_data.drop('cost', axis=1) # Features
y = processed_data['cost'] # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svr', SVR())
])

param_distributions = {
    'svr__C': [0.1, 1, 10],
    'svr__epsilon': [0.01, 0.1, 0.2, 0.5],
    'svr__kernel': ['rbf', 'linear']
}

# Initialize the Randomized Search model
randomized_search = RandomizedSearchCV(pipeline, param_distributions, n_iter=10, cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)

# Fit the Randomized Search model
randomized_search.fit(X_train, y_train)

print("Best parameters found: ", randomized_search.best_params_)
print("Best CV score: ", -randomized_search.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


Fatal Python error: config_get_locale_encoding: failed to get the locale encoding: nl_langinfo(CODESET) failed
Python runtime state: preinitialized

Fatal Python error: config_get_locale_encoding: failed to get the locale encoding: nl_langinfo(CODESET) failed
Python runtime state: preinitialized

Fatal Python error: config_get_locale_encoding: failed to get the locale encoding: nl_langinfo(CODESET) failed
Python runtime state: preinitialized

Fatal Python error: config_get_locale_encoding: failed to get the locale encoding: nl_langinfo(CODESET) failed
Python runtime state: preinitialized

Fatal Python error: config_get_locale_encoding: failed to get the locale encoding: nl_langinfo(CODESET) failed
Python runtime state: preinitialized

Fatal Python error: config_get_locale_encoding: failed to get the locale encoding: nl_langinfo(CODESET) failed
Python runtime state: preinitialized



Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


KeyboardInterrupt: 

In [ ]:
svr_model = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
svr_model.fit(X_train, y_train)

In [167]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = svr_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')


Mean Squared Error: 609.3729932390178
R-squared: 0.32555795952975386


In [168]:
print(y[:10])

0    126.62
1     59.86
2     84.16
3     95.78
4     50.79
5     50.79
6     95.78
7     59.86
8     59.86
9     84.16
Name: cost, dtype: float64


In [169]:
print(y_pred[:10])

[111.90774891 112.12684518  92.97793176  76.60310536  88.40995878
 108.56276834  75.73045909  81.31039372 114.28647492 103.44609693]
